In [169]:
import pandas as pd
import os
import json

include_categories = False

dataset = "yelp"

DATASET_DIR = f"/Volumes/Forster Neu/Masterarbeit Data/{dataset}_dataset/"


available_datasets = ["foursquarenyc", "foursquaretky", "yelp", "gowalla", "brightkite", "snowcard"]

In [170]:
def open_big_json(file_path):
    data = []

    # Open the file and read it line by line
    with open(file_path, 'r') as file:
        for line in file:
            # Parse the JSON data and append to the list
            data.append(json.loads(line))

    # Create a DataFrame from the list of records
    df = pd.DataFrame(data)

    # Print the first few rows of the DataFram

    return df

In [171]:
def convert_to_unix_timestamp(df, column_name):
    """
    Convert a column of timestamps in a DataFrame to Unix timestamps.

    Args:
        df (pd.DataFrame): The DataFrame containing the timestamp column.
        column_name (str): The name of the column with timestamps in "%Y-%m-%d %H:%M:%S" format.

    Returns:
        pd.DataFrame: The DataFrame with an additional column for Unix timestamps.
    """
    # Convert the column to datetime objects
    df[column_name] = pd.to_datetime(df[column_name], format="mixed")
    
    # Convert datetime objects to Unix timestamps
    df[f'{column_name}'] = df[column_name].apply(lambda x: x.timestamp())
    
    return df

In [172]:
if dataset == "snowcard":
    checkin_df = pd.read_csv(DATASET_DIR+"TSC_EEL_EXPORT.csv", encoding="latin1", sep=";", header=None, names=["timestamp:float", "user_id:token", "category_id:token", "category_name:token_seq", "name:token_seq", "user_type:token_seq"])
    checkin_df["item_id:token"], item_id = pd.factorize(checkin_df["name:token_seq"])
    user_df = checkin_df[["user_id:token", "user_type:token_seq"]].drop_duplicates(subset=["user_id:token"])
    poi_df = checkin_df[["item_id:token", "name:token_seq", "category_id:token", "category_name:token_seq"]].drop_duplicates(subset=["item_id:token"])
    checkin_df = checkin_df[["user_id:token", "item_id:token", "timestamp:float"]]

elif dataset == "foursquarenyc" or dataset == "foursquaretky":
    checkin_df = pd.read_csv(DATASET_DIR + "foursquare_data.csv", sep=",")
    checkin_df = checkin_df.drop(columns=["timezoneOffset"])
    checkin_df = checkin_df.rename(columns={"venueId": "item_id:token", "venueCategoryId": "category_id:token", "venueCategory": "category_name:token_seq", "userId": "user_id:token", "utcTimestamp": "timestamp:float", "latitude": "lat:float", "longitude": "lon:float"})
    user_df = checkin_df[["user_id:token"]].drop_duplicates()

    poi_df = checkin_df[["item_id:token", "category_id:token", "category_name:token_seq", "lat:float", "lon:float"]].drop_duplicates(subset=["item_id:token"])
    checkin_df = checkin_df[["user_id:token", "item_id:token", "timestamp:float"]]

elif dataset == "gowalla" or dataset == "brightkite":
    checkin_df = pd.read_csv(DATASET_DIR + f"loc-{dataset}_totalCheckins.txt", sep="\t", header=None, names=['user_id:token', 'timestamp:float', 'lat:float', 'lon:float', 'item_id:token'])
    checkin_df = checkin_df[~checkin_df['item_id:token'].isin(["00000000000000000000000000000000", "ede07eeea22411dda0ef53e233ec57ca"])]
    user_df = pd.read_csv(DATASET_DIR + f"loc-{dataset}_edges.txt", sep="\t", header=None, names=['user_id:token', 'friends:token_seq'])
    user_df = user_df.groupby('user_id:token')['friends:token_seq'].apply(lambda x: ','.join(map(str, x))).reset_index()
    user_df.columns = ['user_id:token', 'friends:token_seq']
    poi_df = checkin_df[['item_id:token', "lat:float", "lon:float"]].drop_duplicates(subset="item_id:token")
    checkin_df = checkin_df.drop(columns=["lat:float", "lon:float"])




elif dataset == "yelp":
    poi_df = pd.read_json(DATASET_DIR + "yelp_academic_dataset_business.json", lines=True)
    poi_df = poi_df.loc[poi_df['is_open'] == 1]
    poi_df = poi_df.drop(columns=["review_count", "stars", "hours", "is_open", "city", "state", "postal_code", "attributes", "address"])
    poi_df = poi_df.rename(columns={"latitude": "lat:float", "longitude": "lon:float", "business_id": "item_id:token", "name":"name:token_seq", "categories":"category_name:token_seq"})
    user_df = open_big_json(DATASET_DIR + "yelp_academic_dataset_user.json")
    user_df = user_df.drop(columns=["review_count", "name", "yelping_since", "useful", "funny", "cool", "elite", "fans", "compliment_hot", "average_stars", "compliment_more", "compliment_profile", "compliment_cute", "compliment_list", "compliment_note", "compliment_plain", "compliment_cool", "compliment_funny", "compliment_writer", "compliment_photos"])
    user_df = user_df.rename(columns={"user_id": "user_id:token", "friends": "friends:token_seq"})
    checkin_df = open_big_json(DATASET_DIR + "yelp_academic_dataset_review.json")
    checkin_df = checkin_df.drop(columns=["text", "cool", "stars", "useful", "funny", "review_id"])
    checkin_df = checkin_df.rename(columns={"user_id": "user_id:token", "business_id": "item_id:token", "date": "timestamp:float"})
    

In [173]:
poi_df

,item_id:token,name:token_seq,lat:float,lon:float,category_name:token_seq
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,38.551126,-90.335695,"Shipping Centers, Local Services, Notaries, Ma..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,39.955505,-75.155564,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,40.338183,-75.471659,"Brewpubs, Breweries, Food"
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,36.269593,-87.058943,"Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
6,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,38.627695,-90.340465,"Sporting Goods, Fashion, Shoe Stores, Shopping..."
...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,53.468419,-113.492054,"Nail Salons, Beauty & Spas"
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,36.115118,-86.766925,"Pets, Nurseries & Gardening, Pet Stores, Hobby..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,39.908707,-86.065088,"Shopping, Jewelry, Piercing, Toy Stores, Beaut..."
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,38.782351,-89.950558,"Fitness/Exercise Equipment, Eyewear & Optician..."


In [174]:
print("Number of users, number of POIs", len(checkin_df["user_id:token"].unique()), len(checkin_df["item_id:token"].unique())
)
print("Sparsity:", 1 - len(checkin_df) / (len(checkin_df["user_id:token"].unique()) * len(checkin_df["item_id:token"].unique())))

Number of users, number of POIs 1987929 150346
Sparsity: 0.999976611529346


In [175]:
def filter_df(df, min_reviews_user=15, min_reviews_business=10):
    while True:
        # Filter businesses with at least min_reviews reviews
        business_counts = df["item_id:token"].value_counts()
        business_mask = df['item_id:token'].map(business_counts) >= min_reviews_business
        df_filtered = df.loc[business_mask]

        # Filter users with at least min_reviews reviews
        user_counts = df_filtered['user_id:token'].value_counts()
        user_mask = df_filtered['user_id:token'].map(user_counts) >= min_reviews_user
        df_filtered = df_filtered.loc[user_mask]

        # If the size of the filtered DataFrame didn't change, break the loop
        if df_filtered.shape[0] == df.shape[0]:
            break

        # Update the DataFrame for the next iteration
        df = df_filtered

    return df_filtered

In [176]:
checkin_df_filtered = filter_df(checkin_df, min_reviews_business=10, min_reviews_user=15)

In [177]:
checkin_df_filtered

,user_id:token,item_id:token,timestamp:float
0,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,2018-07-07 22:09:11
4,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,2017-01-14 20:54:15
13,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,2009-10-14 19:57:14
14,4Uh27DgGzsp6PqrH913giQ,otQS34_MymijPTdNBoBdCw,2011-10-27 17:12:05
17,j2wlzrntrbKwyOcOiB3l3w,rBdG_23USc7DletfZ11xGA,2014-08-10 19:41:43
...,...,...,...
6990272,7L7GL5Pi2cf8mbm2Dpw4zw,e_E-jq9mwm7wk75k7Yi-Xw,2022-01-17 22:36:01
6990274,RKPkxOYQlM0BjhM-H6_vAw,X4mouE_cMiwbfyCPZ_K-FA,2015-01-23 23:52:03
6990277,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,2019-12-30 03:56:30
6990278,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,2022-01-19 18:59:27


In [178]:
# Step 1: Calculate the value counts of `business_id`
value_counts = checkin_df_filtered['item_id:token'].value_counts().reset_index()
value_counts.columns = ['item_id:token', 'count']

# Step 2: Normalize the counts y dividing by the maximum value count
max_count = value_counts['count'].max()
value_counts['business_popularity:float'] = value_counts['count'] / max_count

# Step 3: Merge the normalized counts back into the original DataFrame
checkin_df_filtered = checkin_df_filtered.merge(value_counts[['item_id:token', 'business_popularity:float']], on = "item_id:token", how='left')


In [179]:
def user_popularity_sample_calculator(checkin_df_filtered, poi_df, user_df, sep_num):
    # Calculate average popularity per user
    average_popularity_per_user = checkin_df_filtered.groupby('user_id:token')['business_popularity:float'].mean().reset_index()
    average_popularity_per_user.columns = ['user_id:token', 'average_popularity']

    average_popularity_per_user = average_popularity_per_user.sort_values(by="average_popularity", ascending=False)

    
    # Sort by average popularity
    

    # Get top 1000 users
    high_pop_user_df_sample = average_popularity_per_user.head(sep_num)
    
    # Get the middle 1000 users around the median
    median_index = len(average_popularity_per_user) // 2
    start_med_index = max(median_index -int (sep_num/2), 0)
    end_med_index = min(median_index + int(sep_num/2), len(average_popularity_per_user))
    med_pop_user_df_sample = average_popularity_per_user.iloc[start_med_index:end_med_index]
    
    # Get the lowest 1000 users
    low_pop_user_df_sample = average_popularity_per_user.tail(sep_num)

    unique_users = list(set(high_pop_user_df_sample["user_id:token"].tolist() + med_pop_user_df_sample["user_id:token"].tolist() + low_pop_user_df_sample["user_id:token"].tolist()))

    checkin_df_sample = checkin_df_filtered[checkin_df_filtered["user_id:token"].isin(unique_users)]

    checkin_df_sample = checkin_df_sample[checkin_df_sample["user_id:token"].isin(unique_users)]

    # unique_items = checkin_df_sample["item_id:token"].unique()
    # print(len(unique_items))

    user_df_sample = user_df[user_df["user_id:token"].isin(unique_users)]
    poi_df_sample = poi_df[poi_df["item_id:token"].isin(checkin_df_sample["item_id:token"])]

    checkin_df_sample = checkin_df_sample[checkin_df_sample["item_id:token"].isin(poi_df_sample["item_id:token"])]
    



    
    return checkin_df_sample, high_pop_user_df_sample, med_pop_user_df_sample, low_pop_user_df_sample, user_df_sample, poi_df_sample

In [180]:
checkin_df_sample, high_pop_user_df_sample, med_pop_user_df_sample, low_pop_user_df_sample, user_df_sample, poi_df_sample = user_popularity_sample_calculator(checkin_df_filtered, poi_df, user_df, 500)

In [181]:
poi_df_sample


,item_id:token,name:token_seq,lat:float,lon:float,category_name:token_seq
12,il_Ro8jwPlHresjw9EGmBg,Denny's,39.637133,-86.127217,"American (Traditional), Restaurants, Diners, B..."
15,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,39.953949,-75.143226,"Sushi Bars, Restaurants, Japanese"
19,ROeacJQwBeh05Rqg7F6TCg,BAP,39.943223,-75.162568,"Korean, Restaurants"
21,qhDdDeI3K4jy2KyzwFN53w,Barnes & Noble Booksellers,39.913820,-86.105739,"Shopping, Books, Mags, Music & Video, Bookstores"
23,9OG5YkX1g2GReZM0AskizA,Romano's Macaroni Grill,39.476117,-119.789339,"Restaurants, Italian"
...,...,...,...,...,...
150296,lCo7sEuxlfFeT-nAOR-sbA,Apple Store,30.005648,-90.157396,"IT Services & Computer Repair, Mobile Phones, ..."
150315,WXNag-tmudrFHFlfNLogWg,Whole Foods Market,38.641988,-90.261655,"Health Markets, Specialty Food, Food, Grocery"
150321,AM7O0cwkxm6w_e0Q7-f9FQ,Starbucks,39.638245,-86.128069,"Food, Coffee & Tea"
150323,w_4xUt-1AyY2ZwKtnjW0Xg,Bittercreek Alehouse,43.616590,-116.202383,"Bars, Gastropubs, Sandwiches, Nightlife, Resta..."


In [182]:
checkin_df_sample["item_id:token"].nunique()

15592

In [183]:
poi_df_sample

,item_id:token,name:token_seq,lat:float,lon:float,category_name:token_seq
12,il_Ro8jwPlHresjw9EGmBg,Denny's,39.637133,-86.127217,"American (Traditional), Restaurants, Diners, B..."
15,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,39.953949,-75.143226,"Sushi Bars, Restaurants, Japanese"
19,ROeacJQwBeh05Rqg7F6TCg,BAP,39.943223,-75.162568,"Korean, Restaurants"
21,qhDdDeI3K4jy2KyzwFN53w,Barnes & Noble Booksellers,39.913820,-86.105739,"Shopping, Books, Mags, Music & Video, Bookstores"
23,9OG5YkX1g2GReZM0AskizA,Romano's Macaroni Grill,39.476117,-119.789339,"Restaurants, Italian"
...,...,...,...,...,...
150296,lCo7sEuxlfFeT-nAOR-sbA,Apple Store,30.005648,-90.157396,"IT Services & Computer Repair, Mobile Phones, ..."
150315,WXNag-tmudrFHFlfNLogWg,Whole Foods Market,38.641988,-90.261655,"Health Markets, Specialty Food, Food, Grocery"
150321,AM7O0cwkxm6w_e0Q7-f9FQ,Starbucks,39.638245,-86.128069,"Food, Coffee & Tea"
150323,w_4xUt-1AyY2ZwKtnjW0Xg,Bittercreek Alehouse,43.616590,-116.202383,"Bars, Gastropubs, Sandwiches, Nightlife, Resta..."


In [184]:
high_pop_user_df_sample

,user_id:token,average_popularity
30219,YhdXYhX2VC4SgixBH3ernQ,0.382032
29649,Y3SuYW73lQjqDhgi3tg3Fw,0.362980
51581,wlQM5HVrRq4Y38R651OT_w,0.349215
17038,IziIV6r8Y8Zuoy-2ee4xqQ,0.336167
53246,yexCJ1nAbrGAYKKUvUogPA,0.332450
...,...,...
35022,dTS96BwskmWbiY9MeROKsg,0.202151
22456,P_fQGyQRAdU1diKxhnDB7A,0.202125
12869,E0fG15nooadD2pqD1sERAA,0.202034
21531,OTgf_lu0FMK68YUR3X_3jg,0.201932


In [185]:
def id_factorizer(checkin_df_sample, high_pop_user_df_sample, med_pop_user_df_sample, low_pop_user_df_sample, user_df_sample, poi_df_sample):
    """Overwriting the actual ID with a factorized ID so that we can use the same ID both in RecBole and CAPRI"""
    checkin_df_sample['user_id:token'], user_id_map = pd.factorize(checkin_df_sample['user_id:token'])
    checkin_df_sample['item_id:token'], business_id_map = pd.factorize(checkin_df_sample['item_id:token'])

    # Create mapping dictionaries
    user_id_mapping = {original: i for i, original in enumerate(user_id_map)}
    business_id_mapping = {original: j for j, original in enumerate(business_id_map)}

    high_pop_user_df_sample['user_id:token'] = high_pop_user_df_sample['user_id:token'].map(user_id_mapping)
    med_pop_user_df_sample['user_id:token'] = med_pop_user_df_sample['user_id:token'].map(user_id_mapping)
    low_pop_user_df_sample['user_id:token'] = low_pop_user_df_sample['user_id:token'].map(user_id_mapping)

    user_df_sample['user_id:token'] = user_df_sample['user_id:token'].map(user_id_mapping)
    poi_df_sample['item_id:token'] = poi_df_sample['item_id:token'].map(business_id_mapping)

    return checkin_df_sample, high_pop_user_df_sample, med_pop_user_df_sample, low_pop_user_df_sample, user_df_sample, poi_df_sample

In [186]:
checkin_df_sample, high_pop_user_df_sample, med_pop_user_df_sample, low_pop_user_df_sample, user_df_sample, poi_df_sample = id_factorizer(checkin_df_sample, high_pop_user_df_sample, med_pop_user_df_sample, low_pop_user_df_sample, user_df_sample, poi_df_sample)

/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_46414/2675449884.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_df_sample['user_id:token'] = user_df_sample['user_id:token'].map(user_id_mapping)
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_46414/2675449884.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poi_df_sample['item_id:token'] = poi_df_sample['item_id:token'].map(business_id_mapping)


In [187]:
checkin_df_sample["item_id:token"].nunique()

15592

In [188]:
def data_saver(df, filename, framework):
    if not os.path.exists(DATASET_DIR + "processed_data_" + framework):
        os.makedirs(DATASET_DIR + "processed_data_" + framework)
    
    df.to_csv(DATASET_DIR + "processed_data_" + framework + "/" + filename + ".csv")
    print("Data saved as " + framework + filename + ".csv")
    

In [189]:
# first of all saving data for cornac
data_saver(checkin_df_sample, "user_events", "cornac")
data_saver(high_pop_user_df_sample, "high_pop_user_sample", "cornac")
data_saver(med_pop_user_df_sample, "medium_pop_user_sample", "cornac")
data_saver(low_pop_user_df_sample, "low_pop_user_sample", "cornac")


Data saved as cornacuser_events.csv
Data saved as cornachigh_pop_user_sample.csv
Data saved as cornacmedium_pop_user_sample.csv
Data saved as cornaclow_pop_user_sample.csv


In [190]:
def data_saver_recbole(df, framework, suffix):
    
    if not os.path.exists(DATASET_DIR + "processed_data_" + framework):
        os.makedirs(DATASET_DIR + "processed_data_" + framework)

    df.to_csv(f"{DATASET_DIR}processed_data_{framework}/{dataset}_sample.{suffix}", sep="\t", index=False)

In [191]:
checkin_df_sample['review_id:token'] = range(1, len(checkin_df_sample) + 1)
# Step 1: Group by user_id and business_id and count check-ins
checkin_df_sample['checkin_count:float'] = checkin_df_sample.groupby(['user_id:token', 'item_id:token'])['item_id:token'].transform('count')


In [192]:
checkin_df_sample

,user_id:token,item_id:token,timestamp:float,business_popularity:float,review_id:token,checkin_count:float
77,0,0,2005-07-10 17:20:44,0.031880,1,1
160,1,1,2012-01-03 16:29:01,0.029067,2,1
205,2,2,2008-12-03 20:05:18,0.020159,3,1
350,3,3,2018-01-09 22:21:22,0.008439,4,1
415,4,4,2016-07-03 20:08:28,0.157993,5,1
...,...,...,...,...,...,...
2172656,240,14715,2020-01-14 19:33:00,0.010783,42988,1
2172787,818,14487,2016-11-22 00:10:14,0.013596,42989,1
2172886,1238,14613,2022-01-10 04:02:02,0.073605,42990,1
2172899,1013,15591,2015-09-20 22:24:47,0.012658,42991,1


In [193]:
checkin_df_sample = convert_to_unix_timestamp(checkin_df_sample, "timestamp:float")

In [194]:
checkin_df_sample.sort_values(by="checkin_count:float", ascending=False)
# very important: keeping the duplicate check-ins for the context aware recommendation to have the timestamps saved
checkin_df_timestamps = checkin_df_sample.copy()

# very important: dropping duplicate check-ins 
checkin_df_sample = checkin_df_sample.drop_duplicates(subset=["user_id:token", "item_id:token"], keep="first")

In [195]:
user_df_sample = user_df[["user_id:token"]]

In [196]:
data_saver_recbole(checkin_df_sample, "recbole", "inter")
data_saver_recbole(user_df_sample, "recbole", "user")
data_saver_recbole(poi_df_sample, "recbole", "item")

In [197]:
poi_df_sample

,item_id:token,name:token_seq,lat:float,lon:float,category_name:token_seq
12,753,Denny's,39.637133,-86.127217,"American (Traditional), Restaurants, Diners, B..."
15,924,Tuna Bar,39.953949,-75.143226,"Sushi Bars, Restaurants, Japanese"
19,1294,BAP,39.943223,-75.162568,"Korean, Restaurants"
21,469,Barnes & Noble Booksellers,39.913820,-86.105739,"Shopping, Books, Mags, Music & Video, Bookstores"
23,689,Romano's Macaroni Grill,39.476117,-119.789339,"Restaurants, Italian"
...,...,...,...,...,...
150296,14743,Apple Store,30.005648,-90.157396,"IT Services & Computer Repair, Mobile Phones, ..."
150315,14538,Whole Foods Market,38.641988,-90.261655,"Health Markets, Specialty Food, Food, Grocery"
150321,15501,Starbucks,39.638245,-86.128069,"Food, Coffee & Tea"
150323,15044,Bittercreek Alehouse,43.616590,-116.202383,"Bars, Gastropubs, Sandwiches, Nightlife, Resta..."


In [198]:
checkin_df_timestamps = checkin_df_timestamps[["user_id:token", "item_id:token", "timestamp:float"]] # FINAL
checkins_capri_train_test_tune = checkin_df_sample[["user_id:token", "item_id:token", "timestamp:float", "checkin_count:float"]]
try:
    poi_df_sample_capri = poi_df_sample[["item_id:token", "lat:float", "lon:float"]] # FINAL
except KeyError: # in the snowcard data the coordinates are not given
    poi_df_sample_capri = poi_df_sample[["item_id:token"]]
datasize_capri = pd.DataFrame(data={"num_users" : [len(checkins_capri_train_test_tune["user_id:token"].unique())], "num_items" : [len(checkins_capri_train_test_tune["item_id:token"].unique())]}) # FINAL

In [199]:
# splitting the data into train, test, and tune
checkins_capri_train_test_tune = checkins_capri_train_test_tune.sort_values(by=["user_id:token", "timestamp:float"])
checkins_capri_train_test_tune = checkins_capri_train_test_tune[["user_id:token", "item_id:token", "checkin_count:float"]]

# Split the data
train_list = []
val_list = []
test_list = []

for user, group in checkins_capri_train_test_tune.groupby('user_id:token'):
    n = len(group)
    train_end = int(n * 0.65)
    val_end = int(n * 0.80)
    
    train_list.append(group.iloc[:train_end])
    val_list.append(group.iloc[train_end:val_end])
    test_list.append(group.iloc[val_end:])

# Combine lists into DataFrames
train_df = pd.concat(train_list)
val_df = pd.concat(val_list)
test_df = pd.concat(test_list)



# Check the splits

# FINAL 6-8
print("Train Set:")
print(train_df.head())
print("\nValidation Set:")
print(val_df.head())
print("\nTest Set:")
print(test_df.head())


Train Set:
         user_id:token  item_id:token  checkin_count:float
1112150              0           8095                    1
1742733              0          12527                    1
446265               0           3265                    1
225035               0           1657                    1
1529053              0          10979                    1

Validation Set:
         user_id:token  item_id:token  checkin_count:float
1707931              0          11764                    1
1308970              0           9440                    1
1310994              0           9478                    1
674355               0           4882                    1
2723                 0             54                    1

Test Set:
         user_id:token  item_id:token  checkin_count:float
1067465              0           7252                    1
759302               0           5645                    1
2063660              0          14329                    1
1789814          

In [200]:
def datasaver_capri(df, filename):
    
    if not os.path.exists(DATASET_DIR + "processed_data_capri"):
        os.makedirs(DATASET_DIR + "processed_data_capri")
    
    df.to_csv(DATASET_DIR + "processed_data_capri/" + filename + ".txt", sep='\t', index=False, header=False)
    print("Data saved as " + filename + ".txt")
    



In [201]:
# adding a category column
if include_categories is True:
    if dataset == "yelp":
        # Split the 'category_name' column by commas
        poi_df_sample['category_name_unstacked:token_seq'] = poi_df_sample['category_name:token_seq'].str.split(', ')

        # Unstack the categories into multiple rows
        category_df_sample = poi_df_sample.explode('category_name_unstacked:token_seq')
        category_counts = category_df_sample["category_name_unstacked:token_seq"].value_counts()
        category_mask = category_df_sample["category_name_unstacked:token_seq"].map(category_counts) >= 25
        category_df_sample_filtered = category_df_sample.loc[category_mask]
        category_df_sample_filtered["category_id:token"], category_id = pd.factorize(category_df_sample_filtered["category_name_unstacked:token_seq"])
        category_df_sample_filtered.dropna(inplace=True)
        datasize_capri = pd.DataFrame(data={"num_users" : [len(checkins_capri_train_test_tune["user_id:token"].unique())], "num_items" : [len(checkins_capri_train_test_tune["item_id:token"].unique())], "num_categories" : [len(category_df_sample_filtered["category_id:token"].unique())]}) # FINAL
        datasaver_capri(category_df_sample_filtered, "poiCategories")


    elif dataset == "foursquarenyc" or dataset == "foursquaretky":
        poi_df_sample["category_id:token"], category_id = pd.factorize(poi_df_sample["category_name:token_seq"])
        datasize_capri = pd.DataFrame(data={"num_users" : [len(checkins_capri_train_test_tune["user_id:token"].unique())], "num_items" : [len(checkins_capri_train_test_tune["item_id:token"].unique())], "num_categories" : [len(poi_df_sample["category_id:token"].unique())]})
        poi_df_categories = poi_df_sample[["item_id:token", "category_id:token"]]
        datasaver_capri(poi_df_categories, "poiCategories")

    elif dataset == "snowcard":
        datasize_capri = pd.DataFrame(data={"num_users" : [len(checkins_capri_train_test_tune["user_id:token"].unique())], "num_items" : [len(checkins_capri_train_test_tune["item_id:token"].unique())], "num_categories" : [len(poi_df_sample["category_id:token"].unique())]})
        poi_df_categories = poi_df_sample[["item_id:token", "category_id:token"]]
        datasaver_capri(poi_df_categories, "poiCategories")
    

In [202]:
datasaver_capri(checkin_df_timestamps, "checkins")
datasaver_capri(datasize_capri, "dataSize")
datasaver_capri(poi_df_sample_capri, "poiCoos")
datasaver_capri(train_df, "train")
datasaver_capri(val_df, "tune")
datasaver_capri(test_df, "test")

Data saved as checkins.txt
Data saved as dataSize.txt
Data saved as poiCoos.txt
Data saved as train.txt
Data saved as tune.txt
Data saved as test.txt


In [203]:
train_df

,user_id:token,item_id:token,checkin_count:float
1112150,0,8095,1
1742733,0,12527,1
446265,0,3265,1
225035,0,1657,1
1529053,0,10979,1
...,...,...,...
1791239,1497,12950,1
2048081,1497,14087,1
1203418,1497,8079,1
975832,1497,6356,1


In [204]:
data_saver_recbole(train_df, "recbole_debias", "train.inter")
data_saver_recbole(test_df, "recbole_debias", "test.inter")
data_saver_recbole(val_df, "recbole_debias", "valid.inter")